In [ ]:
# Tests for Horizon attributes
import os
import sys
from datetime import date
import warnings
import numpy as np

warnings.filterwarnings('ignore')

sys.path.append('../../../seismiqb')

from seismiqb import Field, SeismicGeometry, Horizon

In [ ]:
# Workspace constants
TESTS_ROOT_DIR = './'

# Visualization parameters
SCALE = 1

# Tests parameters
ATTRS_LIST = ['amplitudes', 'depths', 'instant_phases',
              'instant_amplitudes', 'fourier', 'wavelet', 'masks']

In [ ]:
# Tests parameters
OUTPUT_DIR = os.path.join(TESTS_ROOT_DIR, "horizon_test_files")

CUBE_PATH = os.path.join(OUTPUT_DIR, f"test_cube.sgy")
HORIZON_PATH = os.path.join(OUTPUT_DIR, f"test_horizon")

In [ ]:
%%time
geometry = SeismicGeometry(CUBE_PATH)
field = Field(geometry)

# Init from_file
horizon = Horizon(HORIZON_PATH, field=field)

# Make it pretty:
horizon.filter()

# Operations with FILL_VALUE

In [ ]:
%%time
horizon_copy = horizon.__copy__()

# Change fill_value to a new one
new_fill_value = -10

# New and old fill_values amount before filling
n_fill_value = np.sum(horizon_copy.matrix == new_fill_value)
n_absent = np.sum(horizon_copy.matrix == horizon_copy.FILL_VALUE)

horizon_copy.matrix_fill_to_num(matrix=horizon_copy.matrix, value=new_fill_value)

# New fill_values amount after filling
new_n_fill_value = np.sum(horizon_copy.matrix == new_fill_value)

assert new_n_fill_value == n_fill_value + n_absent, "matrix_fill_to_num test failed"

In [ ]:
%%time
# Reverse filling: new fill_value to old
horizon_copy.matrix_num_to_fill(matrix=horizon_copy.matrix, value=new_fill_value)

old_n_fill_value = np.sum(horizon_copy.matrix == horizon_copy.FILL_VALUE)

assert old_n_fill_value == new_n_fill_value, "matrix_num_to_fill test failed"

# Matrix operations

In [ ]:
%%time
# Normalize
# Min-max
horizon_copy = horizon.__copy__()

normalized_matrix = horizon_copy.matrix_normalize(matrix=horizon_copy.full_matrix, mode="min-max")

error_msg = "min-max matrix normalization test failed"
assert np.isclose(np.nanmin(normalized_matrix[horizon_copy.mask]), 0), error_msg
assert np.isclose(np.nanmax(normalized_matrix[horizon_copy.mask]), 1), error_msg

# Mean-std
horizon_copy = horizon.__copy__()

normalized_matrix = horizon_copy.matrix_normalize(matrix=horizon_copy.full_matrix, mode="mean-std")

error_msg = "mean-std matrix normalization test failed"
assert np.isclose(np.nanmean(normalized_matrix[horizon_copy.mask]), 0), error_msg
assert np.isclose(np.nanstd(normalized_matrix[horizon_copy.mask]), 1), error_msg

# Properties

In [ ]:
%%time
_ = horizon.binary_matrix
_ = horizon.mask
_ = horizon.borders_matrix
_ = horizon.boundaries_matrix
_ = horizon.filled_matrix

# Get attributes

In [ ]:
%%time
sides = [-1, 1]

for side in sides:
    _ = horizon.get_zerocrossings(side=side)

    print(f"{side} side was successfully checked.")

In [ ]:
%%time
_ = horizon.get_median_diff_map()
_ = horizon.get_gradient_map()
_ = horizon.get_spikes_map()

# Loading attributes

In [ ]:
%%time
metrics = horizon.load_attribute('metrics', metric='local_corrs', normalize='min-max')

In [ ]:
%%time
N_COLS = 2

horizon.show(ATTRS_LIST, separate=True, ncols=N_COLS, scale=SCALE)